# **Bài tập thực hành 1: Xây dựng mô hình từ giải thuật SVM trên dữ liệu bệnh tiểu đường. Dữ liệu lấy từ** https://www.kaggle.com/code/tumpanjawat/diabetes-eda-random-forest-hp

## **Các bước thực hiện:**
1. Import thư viện và nạp dữ liệu
2. Khám phá dữ liệu, kiểm tra thông tin các feature
3. Chia dữ liệu thành tập huấn luyện và kiểm tra
4. Xây dựng mô hình SVM
5. Đánh giá độ chính xác của mô hình
6. Tìm tham số kernel tối ưu cho mô hình SVM

### 1. Import thư viện và nạp dữ liệu

In [1]:
# 1. Import thư viện và nạp dữ liệu
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
data = pd.read_csv('data/diabetes_prediction_dataset.csv')
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


### 2. Khám phá dữ liệu, kiểm tra thông tin các feature

In [2]:
# 2. Khám phá dữ liệu, kiểm tra thông tin các feature
data.info()
data.describe()
data['diabetes'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


diabetes
0    91500
1     8500
Name: count, dtype: int64

### 3. Chia dữ liệu thành tập huấn luyện và kiểm tra

In [3]:
# 3. Chia dữ liệu thành tập huấn luyện và kiểm tra
X = data.drop('diabetes', axis=1)
y = data['diabetes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4. Xây dựng mô hình SVM

In [4]:
# 4. Xây dựng mô hình SVM 
from sklearn.preprocessing import LabelEncoder, StandardScaler
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()
for col in X_train_enc.columns:
    if X_train_enc[col].dtype == 'object':
        le = LabelEncoder()
        X_train_enc[col] = le.fit_transform(X_train_enc[col])
        X_test_enc[col] = le.transform(X_test_enc[col])
# Chuẩn hóa dữ liệu số
scaler = StandardScaler()
X_train_enc = scaler.fit_transform(X_train_enc)
X_test_enc = scaler.transform(X_test_enc)
# Giảm số lượng mẫu để huấn luyện nhanh hơn (ví dụ lấy 1000 mẫu)
n_samples = min(1000, X_train_enc.shape[0])
clf = svm.SVC()
clf.fit(X_train_enc[:n_samples], y_train[:n_samples])

SVC()

### 5. Đánh giá độ chính xác của mô hình

In [5]:
# 5. Đánh giá độ chính xác của mô hình 
train_acc = clf.score(X_train_enc[:n_samples], y_train[:n_samples])
val_acc = clf.score(X_test_enc, y_test)
print('Training accuracy:', train_acc)
print('Validation accuracy:', val_acc)

Training accuracy: 0.958
Validation accuracy: 0.95435


### 6. Tìm tham số kernel tối ưu cho mô hình SVM

In [6]:
# 6. Tìm tham số kernel tối ưu cho mô hình SVM 
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
best_svm = None
best_val_acc = -1
best_kernel = None
for kernel in kernels:
    clf = svm.SVC(kernel=kernel)
    clf.fit(X_train_enc[:n_samples], y_train[:n_samples])
    tmp_val_acc = clf.score(X_test_enc, y_test)
    if tmp_val_acc > best_val_acc:
        best_val_acc = tmp_val_acc
        best_svm = clf
        best_kernel = kernel
print(f'Best validation accuracy: {best_val_acc} with kernel: {best_kernel}')

Best validation accuracy: 0.95805 with kernel: linear
